In [128]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()


# api_key = os.environ["GROQ_API_KEY"]

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant")

In [129]:
%pip install jq


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [159]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import JSONLoader
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# Load embedding model here for RAG
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables.history import RunnableWithMessageHistory


def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["title"] = record.get("title")
    metadata["dish_name"] = record.get("dishName")
    # metadata["ingredients"] = record.get("ingredients")['item'] + " " + record.get("ingredients")['amount'] + " " + record.get("ingredients")['unit']
    # metadata["intructions"] = record.get("instructions")["step"] + ". " + record.get("instructions")["instruction"]

    # if "source" in metadata:
    #         source = metadata["source"].split("/")
    #         source = source[source.index("langchain"):]
    #         metadata["source"] = "/".join(source)

    return metadata

file_path = "../static/recipes.json"
loader = JSONLoader(file_path=file_path, jq_schema=".[]", text_content=False, metadata_func=metadata_func)
data = loader.load()
print(data)
# vectorstore = Chroma.from_documents(documents=data, embedding=OpenAIEmbeddings())
# retriever = vectorstore.as_retriever()

[Document(metadata={'source': '/Users/xela/Developer/hackathons/ai-tinkerers/ai-recipes/static/recipes.json', 'seq_num': 1, 'title': 'Spicy Veggie Stir-Fry', 'dish_name': 'Spicy Veggie Stir-Fry'}, page_content='{"title": "Spicy Veggie Stir-Fry", "dishName": "Spicy Veggie Stir-Fry", "ingredients": [{"item": "thin spaghetti", "amount": "1", "unit": "pound"}, {"item": "sesame oil", "amount": "for drizzling", "unit": ""}, {"item": "low-sodium soy sauce", "amount": "1/2 cup plus more for drizzling", "unit": ""}, {"item": "sherry or low-sodium vegetable broth", "amount": "2", "unit": "tablespoons"}, {"item": "brown sugar", "amount": "2", "unit": "tablespoons"}, {"item": "cornstarch", "amount": "2", "unit": "tablespoons"}, {"item": "sriracha", "amount": "2", "unit": "tablespoons"}, {"item": "fresh ginger, minced", "amount": "1", "unit": "tablespoon"}, {"item": "peanut oil", "amount": "3", "unit": "tablespoons"}, {"item": "yellow onion, cut into large chunks", "amount": "1", "unit": "whole"}, 

In [131]:

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
splits = text_splitter.split_documents(docs)
print(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


[Document(metadata={'source': '/Users/xela/Developer/hackathons/ai-tinkerers/ai-recipes/static/recipes.json', 'seq_num': 1, 'title': 'Spicy Veggie Stir-Fry', 'dish_name': 'Spicy Veggie Stir-Fry'}, page_content='{"title": "Spicy Veggie Stir-Fry", "dishName": "Spicy Veggie Stir-Fry", "ingredients": [{"item": "thin spaghetti", "amount": "1", "unit": "pound"}, {"item": "sesame oil", "amount": "for drizzling", "unit": ""}, {"item": "low-sodium soy sauce", "amount": "1/2 cup plus more for drizzling", "unit": ""}, {"item": "sherry or low-sodium vegetable broth", "amount": "2", "unit": "tablespoons"}, {"item": "brown sugar", "amount": "2", "unit": "tablespoons"}, {"item": "cornstarch", "amount": "2", "unit": "tablespoons"}, {"item": "sriracha", "amount": "2", "unit": "tablespoons"}, {"item": "fresh ginger, minced", "amount": "1", "unit": "tablespoon"}, {"item": "peanut oil", "amount": "3", "unit": "tablespoons"}, {"item": "yellow onion, cut into large chunks", "amount": "1", "unit": "whole"}, 

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

from langchain import hub
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = hub.pull("rlm/rag-prompt")

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel({
    "context": retriever, "question": RunnablePassthrough()
}).assign(answer=rag_chain_from_docs)

for chunk in rag_chain_with_source.stream("What recipes do I have?"):
    print(chunk)

In [132]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "You have a persona of Gordan Ramsey."
    "Curate recipes based off of the recipes provided."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [133]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [144]:
system_prompt = (
    "You are a profressional chef and a chef's assistant. You will be given a recipe and you will need to extract the ingredients and the instructions from the recipe. Format the recipes in a easy to read and sequential manner. Add the ingredient amounts to the instructions. DO NOT USE ingredients that are not present to you. If the name of the dish, title, ingredients, and instructions are duplicated ONLY USE ONE. {context}"
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [145]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="context",
)

In [146]:
conversational_rag_chain.invoke(
    {"input": "What recipes do we have?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'We have two recipes:\n\n1. **Spicy Veggie Stir-Fry**\n2. **Tuna Noodle Casserole**'

In [147]:
conversational_rag_chain.invoke(
    {"input": "Give me the instructions and ingredients for a spicy veggie stir-fry."},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Here are the instructions and ingredients for the Spicy Veggie Stir-Fry:\n\n**Spicy Veggie Stir-Fry**\n\n**Ingredients:**\n\n* 1 pound thin spaghetti\n* Sesame oil (for drizzling)\n* 1/2 cup plus more for drizzling low-sodium soy sauce\n* 2 tablespoons sherry or low-sodium vegetable broth\n* 2 tablespoons brown sugar\n* 2 tablespoons cornstarch\n* 2 tablespoons sriracha\n* 1 tablespoon fresh ginger, minced\n* 3 tablespoons peanut oil\n* 1 whole yellow onion, cut into large chunks\n* 1 whole red bell pepper, cut into large chunks\n* 1 whole yellow bell pepper, cut into large chunks\n* 2 cloves garlic, minced\n* 2 medium zucchini, cut into large wedges\n* 15 ounces baby corn, drained and halved crosswise\n* 1 head broccoli, cut into florets\n* For serving: sesame seeds and 2 whole green onions, sliced\n\n**Instructions:**\n\n1. **For the noodles:** Bring a pot of water to a boil. Cook the spaghetti in the boiling water according to the package instructions. Drain, then drizzle with some

In [148]:
conversational_rag_chain.invoke(
    {"input": "Give me the a recipe under 600 calories with a protein content over 18g per serving. Give me the calorie intake."},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Here is a recipe for a Spicy Veggie Stir-Fry with a protein content over 18g per serving and under 600 calories per serving:\n\n**Spicy Shrimp and Veggie Stir-Fry**\n\n**Servings:** 1\n**Calories per serving:** 540\n**Protein per serving:** 22g\n\n**Ingredients:**\n\n* 1/2 pound large shrimp, peeled and deveined (110 calories, 18g protein)\n* 1/2 cup sliced chicken breast (140 calories, 30g protein)\n* 1 tablespoon sesame oil (120 calories)\n* 1/2 cup sliced yellow onion (45 calories)\n* 1/2 cup sliced red bell pepper (45 calories)\n* 2 cloves garlic, minced (10 calories)\n* 1 cup broccoli florets (55 calories)\n* 1 cup sliced mushrooms (15 calories)\n* 1 tablespoon soy sauce (5 calories)\n* 1 tablespoon sriracha sauce (5 calories)\n* Salt and pepper to taste\n\n**Instructions:**\n\n1. **Heat the sesame oil in a large skillet over medium-high heat. Add the chicken breast and cook until browned and cooked through, about 5 minutes. Remove from the skillet and set aside.**\n2. **Add the 